# Tagging and Extraction Using OpenAI functions
개별 요소 vs 리스트(json) 을 반환하도록 하는 테크닉

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

`Tagging` 클래스에서 사용될 문자열로 구성된 두 parameter를 지정합니다.

각각은 description을 포함하고 값으로 할당할 수 있는 문자열이 정해져 있습니다.

In [ ]:
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [ ]:
convert_pydantic_to_openai_function(Tagging)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [ ]:
model = ChatOpenAI(temperature=0)

In [ ]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])

In [ ]:
# tagging_functions을 묶고, 어떤 함수를 사용할지 직접 지정합니다.
model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

In [ ]:
tagging_chain = prompt | model_with_functions

함수가 호출된 결과를 보면, 각 input이 나타내는 `sentiment`와 `language`가 무엇인지 잘 반환되는 것을 확인할 수 있습니다.

In [ ]:
tagging_chain.invoke({"input": "I love langchain"})

In [ ]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

In [ ]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [ ]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [ ]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

## Extraction

Extraction은 tagging과 유사하지만, 여러 개의 정보를 한꺼번에 추출할 수 있습니다.

In [ ]:
from typing import Optional
class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

위에서 정의한 Person 클래스를 Information 클래스에서 리스트로 받고 있습니다.

In [ ]:
class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")

In [ ]:
# pydantic 문법으로 생성된 클래스를 openai 함수 형태로 변환합니다.
convert_pydantic_to_openai_function(Information)

In [ ]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]
# `Information` 함수를 사용하도록 강제합니다.
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

Joe의 정보는 잘 나타나있기 때문에 30살로 정리가 되지만, Martha에 관한 정보는 없는데 0살이라고 표현됩니다.

In [ ]:
extraction_model.invoke("Joe is 30, his mom is Martha")

위와 같은 문제를 방지하기 위해 system 메세지에 정보가 없으면 추측하지 말라고 알려줍니다.

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [ ]:
extraction_chain = prompt | extraction_model

이번에는 Martha의 나이를 0살로 추측하지 않고 비워둔 것을 확인할 수 있습니다.

In [ ]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

보기 좋은 형태로 반환하기 위해 parser를 붙여줍니다.

In [ ]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [ ]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

여러 개의 정보를 담고 있는 리스트를 바로 반환하기 위해 JsonKeyOutputFunctionsParser를 사용합니다.

In [ ]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [ ]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

딕셔너리의 value만 추출된 것을 확인할 수 있습니다.

In [ ]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

## Doing it for real

위에서 배운 tagging을 훨씬 큰 길이의 텍스트에 적용해 봅니다.

예를 들어, 블로그를 불러와서 하위 텍스트로부터 필요한 태그 정보를 추출합니다.

In [ ]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [ ]:
doc = documents[0]

길이가 굉장히 긴 문서이기 때문에 일부분만 추출하여 테스트합니다.

In [ ]:
page_content = doc.page_content[:10000]

In [ ]:
print(page_content[:1000])

summary, language, keywords를 string 형태로 반환하는 함수를 정의합니다.

In [ ]:
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [ ]:
overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [ ]:
# 결과가 깔끔하게 출력됩니다.
tagging_chain.invoke({"input": page_content})

이번에는 좀 더 복잡한 형태의 chain을 만들어 봅니다.

Paper가 Info 안에서 호출되는 형식입니다.

In [ ]:
class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""
    papers: List[Paper]

In [ ]:
paper_extraction_function = [
    convert_pydantic_to_openai_function(Info)
]
extraction_model = model.bind(
    functions=paper_extraction_function, 
    function_call={"name":"Info"}
)
# key_name을 지정해줌으로써 value가 바로 출력되는 것을 볼 수 있습니다.
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [ ]:
extraction_chain.invoke({"input": page_content})

위의 결과를 보면 텍스트 내의 모든 정보가 추출되지 않은 것을 알 수 있습니다.

이를 해결하기 위해 template을 다시 작성하고 chain을 새로 구성하겠습니다.

In [ ]:
template = """A article will be passed to you. Extract from it all papers that are mentioned by this article. 

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

In [ ]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [ ]:
extraction_chain.invoke({"input": page_content})

template의 내용에 따라 공백 리스트가 반환되는 것을 확인할 수 잇습니다.

In [ ]:
extraction_chain.invoke({"input": "hi"})

처리해야 하는 텍스트의 길이가 엄청나게 긴 경우, 

텍스트를 여러 덩어리로 쪼갠 뒤 순서대로 처리하고 한꺼번에 결과를 반환할 수도 있습니다.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [ ]:
# 14개로 쪼개집니다.
splits = text_splitter.split_text(doc.page_content)

In [ ]:
len(splits)

In [ ]:
# 리스트 내의 원소들을 하나의 리스트로 펼치기 위한 함수입니다.
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [ ]:
flatten([[1, 2], [3, 4]])

In [ ]:
print(splits[0])

In [ ]:
# 여러 원소를 리스트로 입력받은 뒤 한꺼번에 처리하기 위해 RunnableLambda를 사용합니다.
from langchain.schema.runnable import RunnableLambda

In [ ]:
prep = RunnableLambda(
    lambda x: [{"input": doc} for doc in text_splitter.split_text(x)]
)

In [ ]:
prep.invoke("hi")

In [ ]:
# prep에 전달/반환되는 것이 리스트이기 때문에 map 함수를 사용하여 개별 요소를 처리하고 리스트로 반환합니다.
chain = prep | extraction_chain.map() | flatten

In [ ]:
chain.invoke(doc.page_content)